# Assignment 8 - k-Nearest Neighbors (kNNs)
(20 points)

### Add your name(s) and EIDs below
- Student Name: Siem Yonas
- Student UT EID: sby237
- Partner Name:
- Partner UT EID:

### Grading clarifications
1. Before you submit:
    - **Please do not change or remove any variable names** we give you for your answer to each question. You can use other variables if you wish, but the final answer must be stored in that variable.
    - **Please don't use any other libraries** besides those provided in `requirements.txt`, and those [built in to Python](https://docs.python.org/3.8/library/index.html).
2. We will also be double-checking the autograder, so you won't lose points if your formatting causes it to deduct points (but please try to follow the points under 1).
3. After submitting, you may see "All test cases passed!" for Public Tests. Please note that this doesn't mean you have correctly answered every question, as there are no public tests. We will be using a series of hidden tests to verify your answers are correct.

# k-Nearest Neighbors
For this assignment, we are going explore one new classification technique: k nearest neighbors.

We are using a different version of the Melbourne housing data set from earlier in the semester, split into training and testing sets for you. Our goal is to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

At the end of this homework, you will understand how to build and use a kNN model, and improve your data cleaning and data preparation skills. 

In [1]:
# These are the libraries you will use for this assignment.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
%matplotlib inline

In [2]:
# Start off by loading the training dataset.
df_melb = pd.read_csv('melb_data_train.csv')

## Q1 

**Fix our "Date" column to be numeric**: If we inspect our dataframe `df_melb` using the `dtypes` property, we see that the column `Date` is an `object`.  However, we think this column might contain useful information, so your goal is to convert it to [Unix time](https://en.wikipedia.org/wiki/Unix_time).

Unix time is the number of secconds since a fixed time known as the "Unix epoch", which is midnight on January 1st, 1970. For example, the Unix time for March 10th, 2023 is 1,678,474,369 seconds.

- **Use only the libraries imported above** imported libraries to create a new column `UnixTime`. 
    - Be careful, the date strings in the file might have some non-uniform formatting that you have to fix first.  
- Print out the min and max epoch time to check your work.  
- Drop the original `Date` column. 

The Python [reference for time](https://docs.python.org/3/library/time.html) can help you with your conversion to Unix time.

(**3 points**)

In [3]:
# For reference, here are the data types of each column.
df_melb.dtypes

Rooms             int64
Type             object
Price             int64
Date             object
Distance        float64
Postcode          int64
Bathroom          int64
Car             float64
Landsize          int64
BuildingArea    float64
YearBuilt       float64
dtype: object

<!-- BEGIN QUESTION -->



In [4]:
def standardize_date(date_string):
    """Standardize a date string to a standard format.

    Rules:
    - You can assume the input string is of the form day/month/year.
    - Fixed date strings should be of the form DD/MM/YYYY. If a day is
      one digit, append zeros.
    - If the input string's year is two digits (e.g. 02), assume
      the year is in the 2000s (e.g. 2002).
    """
    day, month, year = map(int, date_string.split("/"))
    
    if year < 100:
        year = 2000 + year
    
    fixed_date_string = f"{day:02d}/{month:02d}/{year:02d}"
    return fixed_date_string

def replace_date_with_unix(df):
    """Given a Melbourne dataset dataframe, replace the Date column
    with a UnixTime column.

    Hint: Call standardize_date within this function.
    """
    # Standardize the date column.
    Date = df.Date.apply(standardize_date)
    
    # Create the UnixTime column
    date_to_unix = lambda x: calendar.timegm(time.strptime(x, "%d/%m/%Y"))
    df = df.assign(UnixTime=Date.apply(date_to_unix))
    
    # Drop the date column.
    df = df.drop("Date", axis=1)
    return df

In [5]:
df_melb_q1 = replace_date_with_unix(df_melb)

# Print the cleaned UnixTime values.
print('Min UnixTime:', df_melb_q1['UnixTime'].min())
print('Max UnixTime:', df_melb_q1['UnixTime'].max())

Min UnixTime: 1454544000
Max UnixTime: 1506124800


<!-- END QUESTION -->

## Q2 

**Use imputation to fill in missing values**: kNN doesn't work when some attributes are not present, so we must fill in all the missing values in `df_melb` with something. As a simple estimate, we will fill in missing values with the **mean** of that value/column.

What we're trying to classify ('h'ome/'d'u'plex/'t'ownhouse), also knonw as the **target**, is store in the `Type` column. We define a variable `target_col` which lets you automatically infer which column is the target. During imputation, we should skip this target column.

- Use `df_melb_q1`, i.e. the result from Q1.
- Save the mean of each column in a dictionary `dict_imputation`. Keys are an attribute's column name, and values are that attribute's mean.
- Use `dict_imputation` to imputate the missing values in `df_melb_q1`.
- Store the imputated dataframe in `df_melb_q2`.

(**3 points**)

<!-- BEGIN QUESTION -->



In [6]:
def build_imputation_dict(df, target_col):
    """Collect the mean values of each column, excluding NaN values
    and the target column.
    """

    # Get the mean value of each column.
    dict_imputation = df.drop(columns=target_col).mean().to_dict()
    return dict_imputation

def imputate(df, dict_imputation, target_col):
    """Imputate a dataframe, replacing missing values with those
    given in dict_imputation. Do not imputate target_col."""
    df = df.fillna(dict_imputation)
    
    return df

In [7]:
# Define the target column as a string
target_col = "Type"

# Collect imputation values
dict_imputation = build_imputation_dict(df_melb_q1, target_col)

# Imputate the dataframe
df_melb_q2 = imputate(df_melb_q1, dict_imputation, target_col)

In [8]:
# Check your results
dict_imputation

{'Rooms': 2.710769230769231,
 'Price': 941972.2953846154,
 'Distance': 10.206256410256408,
 'Postcode': 3110.873846153846,
 'Bathroom': 1.4543589743589744,
 'Car': 1.4938398357289528,
 'Landsize': 514.2184615384615,
 'BuildingArea': 131.379476861167,
 'YearBuilt': 1971.0204429301534,
 'UnixTime': 1485036288.0}

In [9]:
# Check your results
df_melb_q2.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,UnixTime
0,3,t,732000,5.6,3101,1,1.0,904,110.000000,1980.000000,1469491200
1,3,h,1001000,12.6,3020,1,5.0,879,131.379477,1971.020443,1488585600
2,2,u,605000,7.4,3185,1,1.0,722,131.379477,1970.000000,1462579200
3,3,h,757500,18.8,3170,2,1.0,145,131.379477,1971.020443,1497657600
4,4,h,721000,17.9,3082,2,2.0,603,131.379477,1971.020443,1505520000


<!-- END QUESTION -->

## Q3

**Normalize all attributes to be between [0,1]**: Normalize all the attribute columns in `df_melb_q2` so they have a value between zero and one (inclusive). 

To do this, we will build a dictionary `dict_normalize`, with column names for keys and (min, max) tuples for values, which are the min (resp. max) value found in the dataframe for that column. Just like in Q2, we do not normalize the target column.

After creating `dict_normalize`, we will use it to normalize each column and generate a new dataframe, `df_melb_q3`. The resulting dataframe is now your model that you can use to classify new data points.

- Use `df_melb_q2`, i.e. the result from Q2.
- Save the minimum and maximum values of each column in a dictionary `dict_normalize`. Keys are an attribute's column name, and values are a (min, amx) tuple for that column,
- Use `dict_normalize` to normalize the missing values in `df_melb_q2`.
- Store the imputated dataframe in `df_melb_q3`.

(**3 points**)

<!-- BEGIN QUESTION -->



In [10]:
def build_normalization_dict(df, target_col):
    """Collect the (min, max) values of each column, except the
    target column.
    """
    
    # Get the min and max values of each column.
    stats = df.drop(columns=target_col).agg(['min', 'max']).to_dict()
    
    dict_normalize = {k: tuple(v.values()) for k, v in stats.items()}
    
    return dict_normalize

def normalize(df, dict_normalize, target_col):
    """Normalize a dataframe, setting all values to the range [0, 1]
    using (min, max) values in dict_normalize. Do not normalize target_col."""
    
    normalized = {k: (df[k] - v[0])/(v[1] - v[0]) for k,v in dict_normalize.items()}
    
    df = df.assign(**normalized)
    
    return df

In [11]:
# Define the target column as a string
target_col = "Type"

# Collect normalization values
dict_normalize = build_normalization_dict(df_melb_q2, target_col)

# Normalize the dataframe
df_melb_q3 = normalize(df_melb_q2, dict_normalize, target_col)

In [12]:
# Check your results
dict_normalize

{'Rooms': (1, 7),
 'Price': (210000, 5020000),
 'Distance': (0.7, 47.3),
 'Postcode': (3000, 3810),
 'Bathroom': (0, 5),
 'Car': (0.0, 8.0),
 'Landsize': (0, 41400),
 'BuildingArea': (0.0, 3558.0),
 'YearBuilt': (1850.0, 2016.0),
 'UnixTime': (1454544000, 1506124800)}

In [13]:
# Check your results
df_melb_q3.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,UnixTime
0,0.333333,t,0.108524,0.105150,0.124691,0.2,0.125,0.021836,0.030916,0.783133,0.289782
1,0.333333,h,0.164449,0.255365,0.024691,0.2,0.625,0.021232,0.036925,0.729039,0.659966
2,0.166667,u,0.082121,0.143777,0.228395,0.2,0.125,0.017440,0.036925,0.722892,0.155779
3,0.333333,h,0.113825,0.388412,0.209877,0.4,0.125,0.003502,0.036925,0.729039,0.835846
4,0.500000,h,0.106237,0.369099,0.101235,0.4,0.250,0.014565,0.036925,0.729039,0.988275


<!-- END QUESTION -->

## Q4 

**Prepare the test data**: Load in `melb_data_test.csv` and repeat the steps in Q1, Q2, and Q3 (unix time, imputation, and normalization).

(**1 point**)

<!-- BEGIN QUESTION -->



In [14]:
# Load the test dataframe
df_melb_test = pd.read_csv("melb_data_test.csv")

# Clean the dates, add unix time
df_melb_test = replace_date_with_unix(df_melb_test)

# Imputate the dataframe
target_col = "Type"
dict_imputation_test = build_imputation_dict(df_melb_test, target_col)
df_melb_test = imputate(df_melb_test, dict_imputation_test, target_col)

# Normalize the dataframe
dict_normalize_test = build_normalization_dict(df_melb_test, target_col)
df_melb_test = normalize(df_melb_test, dict_normalize_test, target_col)

In [15]:
# Check your results
df_melb_test.head()

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,UnixTime
0,0.166667,t,0.183188,0.263736,0.083732,0.5,0.166667,0.041418,0.246521,0.967742,0.835846
1,0.333333,h,0.407216,0.197802,0.145933,0.0,0.333333,0.182397,0.290426,0.624069,0.425461
2,0.666667,h,0.984140,0.129121,0.315789,0.5,0.333333,0.122859,0.290426,0.624069,0.345059
3,0.333333,h,0.206979,0.244505,0.055024,0.0,0.166667,0.118080,0.228628,0.645161,0.355109
4,0.333333,h,0.191118,1.000000,0.449761,0.0,0.333333,0.166268,0.228628,0.564516,0.871022


<!-- END QUESTION -->

## Q5

**Write the kNN classifier function**: Your function, `predict_knn` will take in the following four parameters:
- Training dataframe `df_train`
- Hyperparameter `k`
- Testing sample `test_sample` (one row of the DataFrame, we can generate it using `iloc` or `iterrows`).
- Target column string `target_col`, which defines the variable we want to predict.

It will predict which class `test_sample` belongs to, based on the `k` nearest neighbors to the sample.

- We assume `df_train` is normalized/imputated, contains all attributes, and also contains the target column.
- Likewise, we assume `test_sample` is normalized/imputated and contains all attributes. (But, it does not have to have the target column).

*Hint*: To find the distance between the test sample and any element of the training dataset, you may use the [L2 norm function from numpy](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).

(**5 points**)

<!-- BEGIN QUESTION -->



In [16]:
def predict_knn(df_train: pd.DataFrame, k: int, 
                test_sample: pd.Series, target_col: str):
    """Use the k-nearest neighbors algorithm to predict the class of a test-sample,
    given a training set.
    
    Parameters:
        df_train:    DataFrame of training samples
        k:           Number of neighbors to consider
        test_sample: Single evaluation sample
        target_col:  Name of the target variable (column)

    Returns:
        prediction: Predicted class of the test sample using kNN.
    """
    # Extract parameters from sample
    X_test = test_sample.drop(target_col)
    
    # Calculate distance
    dist = df_train.apply(lambda x : np.linalg.norm( x.drop(index=target_col) - X_test), axis=1)
    df_train = df_train.assign(Distance=dist)
    
    # Find k nearest neighbors
    df_train = df_train.sort_values(by='Distance').reset_index(drop=True)
    df_k = df_train[0:k]
    
    prediction = df_k[target_col].value_counts().index[0] 
    return prediction

<!-- END QUESTION -->

## Q6 

**Compute the accuracy using different k values**: For each value of $k$ in the set $\{1,3,13,25,50,100\}$, compute the kNN prediction for each oberservation in the test set, and the overall accuracy of the classifier.  Plot the accuracy as a function of $k$.

- Use your imputed, normalize training dataframe (`df_melb_q3`).
- Use your imputed, normalized testing dataframe (`df_melb_test`).
- Have an outer loop over the k-values, and an inner loop computing the prediction for each testing sample under that k-value.

Which value of $k$ would you choose? Why?

(This can take a while to run; probably at least 5-10 minutes.)

(**5 points - 3 for implementation, 1 for plot, 1 for description**).

(**This question will be manually graded.**)

$k=13$ should be chosen, as it has the highest accuracy value under majority-voting kNN classification fitting (83%).  


<!-- BEGIN QUESTION -->



In [17]:
# Sweep over the k-values. Place your accuracies for each k-value in acc_k.
poss_k = [1, 3, 13, 25, 50, 100]
acc_k = []

target_col = "Type"

# Your code goes below.
for k in poss_k:
    # Inner "loop"
    predictions = df_melb_test.apply(lambda x: predict_knn(df_melb_q3, k, x, target_col), axis=1)
    
    correct = np.sum(predictions == df_melb_test[target_col])
    acc_k.append(correct/len(predictions))
    
    print(f"k = {k:3}, Acc = {acc_k[-1]:0.8f}")
    

k =   1, Acc = 0.79508197
k =   3, Acc = 0.80327869
k =  13, Acc = 0.82786885
k =  25, Acc = 0.79508197
k =  50, Acc = 0.77868852
k = 100, Acc = 0.77868852


In [18]:
# Plot your accuracies for each k-value.
plt.plot(poss_k, acc_k, 'b-*')
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title('Accuracy on Test Set for Varied k')
plt.grid('on')
plt.show()

<!-- END QUESTION -->

